In [1]:
import pandas as pd
import numpy as np
from __future__ import division

https://people.duke.edu/~ccc14/sta-663/DistributedComputing.html#the-hadoop-mapreduce-workflow

In [2]:
s = 'aabaabcdeda'
xs = map(lambda x: [x, 1], s)
xs

[['a', 1],
 ['a', 1],
 ['b', 1],
 ['a', 1],
 ['a', 1],
 ['b', 1],
 ['c', 1],
 ['d', 1],
 ['e', 1],
 ['d', 1],
 ['a', 1]]

In [3]:
xs = sorted(xs)
xs

[['a', 1],
 ['a', 1],
 ['a', 1],
 ['a', 1],
 ['a', 1],
 ['b', 1],
 ['b', 1],
 ['c', 1],
 ['d', 1],
 ['d', 1],
 ['e', 1]]

In [4]:
ys = []
seen = []
for x in xs:
    if x[0] not in seen:
        seen.append(x[0])
        ys.append([x[0], [x[1]]])
    else:
        ys[-1][1].append(x[1])
ys

[['a', [1, 1, 1, 1, 1]], ['b', [1, 1]], ['c', [1]], ['d', [1, 1]], ['e', [1]]]

In [5]:
from operator import add, itemgetter


In [6]:
for y in ys:
    print y[0], reduce(add, y[1])

a 5
b 2
c 1
d 2
e 1


In [7]:
for y in ys:
    print y[0], reduce(lambda x,y:x+y, y[1])

a 5
b 2
c 1
d 2
e 1


___

In [8]:
l = np.random.randint(1,20,[12,3])
print l.shape
l

(12, 3)


array([[15,  2, 19],
       [14,  1,  9],
       [ 6, 13, 16],
       [ 7,  3,  6],
       [ 7,  9,  9],
       [ 8, 18,  8],
       [12,  1,  2],
       [ 5,  6, 12],
       [17,  7, 19],
       [ 1, 19, 12],
       [ 7,  1, 18],
       [16,  1, 16]])

In [9]:
l[:,[0]]

array([[15],
       [14],
       [ 6],
       [ 7],
       [ 7],
       [ 8],
       [12],
       [ 5],
       [17],
       [ 1],
       [ 7],
       [16]])

In [10]:
l[:,1:]

array([[ 2, 19],
       [ 1,  9],
       [13, 16],
       [ 3,  6],
       [ 9,  9],
       [18,  8],
       [ 1,  2],
       [ 6, 12],
       [ 7, 19],
       [19, 12],
       [ 1, 18],
       [ 1, 16]])

### name, cat, quant, price

walk through一个mapreduce问题，一组数据，四个columns: name, category, # of transactions in 2014, dollar values of transaction in 2014，需要知道每个category的average dollar values per transaction，怎么用mapreduce做，其实就specify一下mapper和reducer的input和output，然后在reducer里求一下平均值什么的

In [11]:
data = np.hstack([l[:,[0]],np.random.randint(0,2,[12,1]), l[:,1:]])
data

array([[15,  0,  2, 19],
       [14,  0,  1,  9],
       [ 6,  1, 13, 16],
       [ 7,  1,  3,  6],
       [ 7,  0,  9,  9],
       [ 8,  1, 18,  8],
       [12,  0,  1,  2],
       [ 5,  1,  6, 12],
       [17,  0,  7, 19],
       [ 1,  0, 19, 12],
       [ 7,  0,  1, 18],
       [16,  0,  1, 16]])

In [74]:
mapper = lambda x: {str(x[1]): x[3]/x[2]}

In [75]:
m = map(mapper, data)
m

[{'0': 9.5},
 {'0': 9.0},
 {'1': 1.2307692307692308},
 {'1': 2.0},
 {'0': 1.0},
 {'1': 0.4444444444444444},
 {'0': 2.0},
 {'1': 2.0},
 {'0': 2.7142857142857144},
 {'0': 0.631578947368421},
 {'0': 18.0},
 {'0': 16.0}]

In [76]:
# np.array(map(mapper, data)).sum(0)

In [77]:
sorted(m)

[{'0': 0.631578947368421},
 {'0': 1.0},
 {'0': 2.0},
 {'0': 2.7142857142857144},
 {'0': 9.0},
 {'0': 9.5},
 {'0': 16.0},
 {'0': 18.0},
 {'1': 0.4444444444444444},
 {'1': 1.2307692307692308},
 {'1': 2.0},
 {'1': 2.0}]

http://www.pythoner.com/46.html

https://www.youtube.com/watch?v=ZrZ6vJGiE8I

In [81]:
m[0]['0']

9.5

In [85]:
def reducer(acc, val):
    try:
        return acc+val['1']
    except:
        return acc

In [89]:
reduce(reducer, m, 0)

5.6752136752136755

In [90]:
m

[{'0': 9.5},
 {'0': 9.0},
 {'1': 1.2307692307692308},
 {'1': 2.0},
 {'0': 1.0},
 {'1': 0.4444444444444444},
 {'0': 2.0},
 {'1': 2.0},
 {'0': 2.7142857142857144},
 {'0': 0.631578947368421},
 {'0': 18.0},
 {'0': 16.0}]

In [106]:
p = {'0':0,'1':0}
p

{'0': 0, '1': 0}

In [118]:
p.items()[0]

('1', 0)

### acc: accumulator  
### val: value

In [122]:
from collections import defaultdict

In [119]:
def reducer2(acc, val):
    key, value = val.items()[0]
    acc[key]+=value
    return acc

In [123]:
reduce(reducer2, m, defaultdict(int))

defaultdict(int, {'0': 67.84586466165413, '1': 5.6752136752136755})

In [142]:
def fun(choose, x, y):
    return {'add': lambda:x+y,'fuck': lambda: x**y}.get(choose, lambda: -(x**-y))()

In [146]:
fun('a',9,11), fun('add',9,11), fun('fuck',9,11), fun('add',9,11)

(-3.186635545324935e-11, 20, 31381059609, 20)

___

In [42]:
import  itertools
scientists =({'name':'Alan Turing', 'age':105, 'gender':'male'},
             {'name':'Dennis Ritchie', 'age':76, 'gender':'male'},
             {'name':'Ada Lovelace', 'age':202, 'gender':'female'},
             {'name':'Frances E. Allen', 'age':84, 'gender':'female'})
grouped = {item[0]:list(item[1])
           for item in itertools.groupby(scientists, lambda x: x['gender'])}
print(grouped)

{'male': [{'gender': 'male', 'age': 105, 'name': 'Alan Turing'}, {'gender': 'male', 'age': 76, 'name': 'Dennis Ritchie'}], 'female': [{'gender': 'female', 'age': 202, 'name': 'Ada Lovelace'}, {'gender': 'female', 'age': 84, 'name': 'Frances E. Allen'}]}


In [43]:
grouped

{'female': [{'age': 202, 'gender': 'female', 'name': 'Ada Lovelace'},
  {'age': 84, 'gender': 'female', 'name': 'Frances E. Allen'}],
 'male': [{'age': 105, 'gender': 'male', 'name': 'Alan Turing'},
  {'age': 76, 'gender': 'male', 'name': 'Dennis Ritchie'}]}

In [3]:
scientists =({'name':'Alan Turing', 'age':105, 'gender':'male'},
             {'name':'Dennis Ritchie', 'age':76, 'gender':'male'},
             {'name':'Ada Lovelace', 'age':202, 'gender':'female'},
             {'name':'Frances E. Allen', 'age':84, 'gender':'female'})
scientists

({'age': 105, 'gender': 'male', 'name': 'Alan Turing'},
 {'age': 76, 'gender': 'male', 'name': 'Dennis Ritchie'},
 {'age': 202, 'gender': 'female', 'name': 'Ada Lovelace'},
 {'age': 84, 'gender': 'female', 'name': 'Frances E. Allen'})

### need python3

In [4]:
from functools import reduce

grouped = reduce(lambda acc, val: {**acc, **{val['gender']: acc[val['gender']]+ [val['name']]}}, 
                 scientists, {'male':[], 'female':[]})
print(grouped)

{'male': ['Alan Turing', 'Dennis Ritchie'], 'female': ['Ada Lovelace', 'Frances E. Allen']}


In [5]:
reduce?

http://www.cnblogs.com/fengfenggirl/p/map-reduce-demo.html

http://www.cnblogs.com/fengfenggirl/p/map-reduce-demo.html